In [1]:
import pandas as pd
import numpy as np
import os
import h2o
from h2o.automl import H2OAutoML

In [2]:
h2o.init()

Checking whether there is an H2O instance running at http://localhost:54321 ..... not found.
Attempting to start a local H2O server...
  Java Version: java version "1.8.0_11"; Java(TM) SE Runtime Environment (build 1.8.0_11-b12); Java HotSpot(TM) 64-Bit Server VM (build 25.11-b03, mixed mode)
  Starting server from /mnt/disk0/home/caihengxing/anaconda3/envs/python37/lib/python3.7/site-packages/h2o/backend/bin/h2o.jar
  Ice root: /tmp/tmpse2oe6j0
  JVM stdout: /tmp/tmpse2oe6j0/h2o_caihengxing_started_from_python.out
  JVM stderr: /tmp/tmpse2oe6j0/h2o_caihengxing_started_from_python.err
  Server is running at http://127.0.0.1:54321
Connecting to H2O server at http://127.0.0.1:54321 ... successful.


H2O_cluster_uptime:,03 secs
H2O_cluster_timezone:,Asia/Shanghai
H2O_data_parsing_timezone:,UTC
H2O_cluster_version:,3.32.1.4
H2O_cluster_version_age:,4 months and 3 days !!!
H2O_cluster_name:,H2O_from_python_caihengxing_p4grmt
H2O_cluster_total_nodes:,1
H2O_cluster_free_memory:,26.67 Gb
H2O_cluster_total_cores:,40
H2O_cluster_allowed_cores:,40
H2O_cluster_status:,"accepting new members, healthy"


In [3]:
train_path = '../data/walmart_recruiting/train.csv'
test_path = '../data/walmart_recruiting/test.csv'

In [4]:
# Load data into H2O
train = h2o.import_file(train_path)
test  = h2o.import_file(test_path)

Parse progress: |█████████████████████████████████████████████████████████| 100%
Parse progress: |█████████████████████████████████████████████████████████| 100%


In [5]:
train.head()

Store,Dept,Date,Weekly_Sales,IsHoliday
1,1,2010-02-05 00:00:00,24924.5,FALSE
1,1,2010-02-12 00:00:00,46039.5,TRUE
1,1,2010-02-19 00:00:00,41595.6,FALSE
1,1,2010-02-26 00:00:00,19403.5,FALSE
1,1,2010-03-05 00:00:00,21827.9,FALSE
1,1,2010-03-12 00:00:00,21043.4,FALSE
1,1,2010-03-19 00:00:00,22136.6,FALSE
1,1,2010-03-26 00:00:00,26229.2,FALSE
1,1,2010-04-02 00:00:00,57258.4,FALSE
1,1,2010-04-09 00:00:00,42960.9,FALSE


In [6]:
y = "Weekly_Sales"

In [7]:
aml2 = H2OAutoML(max_runtime_secs = 7200, seed = 1, project_name = "kaggle_walmart_recruiting")
aml2.train(y = y, training_frame = train)

AutoML progress: |████████████████████████████████████████████████████████| 100%


In [8]:
pred = aml2.predict(test)

stackedensemble prediction progress: |████████████████████████████████████| 100%


In [11]:
sub = pd.read_csv(test_path)
sub = sub[['Store', 'Dept','Date']]

In [12]:
pred = h2o.as_list(pred)

In [13]:
sub[y] = pred['predict']

In [15]:
def cols_concat(df, con_list):
    name = "__".join(con_list)
    df[name] = df[con_list[0]].astype(str)
    for item in con_list[1:]:
        df[name] = df[name] + '_' + df[item].astype(str)
    return df

sub = cols_concat(sub, ['Store', 'Dept','Date'])

In [17]:
del_cols = ['Store', 'Dept','Date']
sub.drop(del_cols, axis=1, inplace=True)

In [21]:
sub.rename({'Store__Dept__Date': 'Id'}, axis=1, inplace=True)

In [22]:
sub = sub[['Id', 'Weekly_Sales']]

In [23]:
sub.to_csv("./h2o_sub_kaggle_walmart_recruiting.csv", index = False)